In [1]:
import sys
import os
sys.path.append('../src')

from semantic_bac_segment.train import UNetTrainer2
from semantic_bac_segment.models.pytorch_attention import AttentionUNet as AttentionUNet
from semantic_bac_segment.models.pytorch_altmodel import UNET as UNet2, DualPathwayUNet, UNET_edges
from semantic_bac_segment.models.pytorch_bilinealup import UNET_bilineal as UNET_bilineal
from semantic_bac_segment.models.pytorch_hed import HEDUNet
from semantic_bac_segment.models.pytorch_unetplusinv import double_UNET
from semantic_bac_segment.models.pytorch_cnnunet import Unet, SegResNet, dilnet, Unet_dynamic

from monai.networks.nets import unet
from semantic_bac_segment.loss_functions import DiceLoss, MultiClassDiceLoss, WeightedBinaryCrossEntropy, weightedDiceLoss
from semantic_bac_segment.data_loader import collate_fn
from semantic_bac_segment.utils import read_cofig
from monai.losses import DiceLoss


In [2]:
from semantic_bac_segment.model_loader import model_loader
import json

with open('/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/train_models.json') as file:
    network_arch=json.load(file)


In [6]:
from semantic_bac_segment.utils import get_device

device=get_device()

for net_i  in network_arch:
    m=model_loader(net_i, device)
    print(m)

UNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (adn): ADN(
            (N): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
        (unit1): Convolution(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (adn): ADN(
            (N): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Sequential(
            (unit

/Users/santiago/miniconda3/envs/img_pytorch/lib/python3.9/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


ValueError: Sequence must have length 3, got 2.

In [7]:
net_i

{'model_name': 'UNETR',
 'model_args': {'in_channels': 1,
  'out_channels': 4,
  'img_size': [256, 256],
  'feature_size': 16,
  'hidden_size': 768,
  'mlp_dim': 3072,
  'num_heads': 12,
  'pos_embed': 'conv',
  'norm_name': 'instance',
  'conv_block': True,
  'res_block': True}}

In [6]:
import tifffile
import numpy as np
import os
path_images='/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/'
path_masks='/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/multiclass_masks'
# Get images
image_files = [os.path.join(path_images, file) for file in os.listdir(path_images) if file.endswith('.tiff')]
mask_files = [os.path.join(path_masks, file) for file in os.listdir(path_masks) if file.endswith('.tiff')]

image_files.sort()
mask_files.sort()

max_width, max_height=2400, 2400

padded_images = []
padded_masks = []
original_dimensions = {}

for image_path, mask_path in zip(image_files, mask_files):
    print(image_path)
    image_name=os.path.basename(mask_path)
    image = tifffile.imread(image_path)
    mask = tifffile.imread(mask_path)
    if image.shape[-1] == 2400 and image.shape[-1] == 2400: 
        height, width = image.shape[:2]
        padded_images.append(image)       
        padded_masks.append(mask)

# Convert the padded images and masks to stacks
image_stack = np.stack(padded_images)
mask_stack = np.stack(padded_masks)

# Print the shape of the stacks
print("Image stack shape:", image_stack.shape)
print("Mask stack shape:", mask_stack.shape)


/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_1.tiff
/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_10.tiff
/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_101.tiff
/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_109.tiff
/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_165.tiff
/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_166.tiff
/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_205.tiff
/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_223.tiff
/Users/santiago/switchdrive/boeck_lab_projects/Semantic_bac_segment/data/source_norm2/coli_mask_frame_232.tiff
/Use

In [16]:
mask_stack=mask_stack/255

In [17]:
# Create a new array with the same shape as multi_class_stack_array
flattened_masks_stack = np.zeros_like(mask_stack)

# Iterate over each channel and assign values based on the position
for c in range(mask_stack.shape[1]):
    flattened_masks_stack[:, c, :, :] = (c + 1) * mask_stack[:, c, :, :]

# Flatten the stack on the channel axis
flattened_masks_stack = np.max(flattened_masks_stack, axis=1)

In [18]:
print(f'shape flatened stack: {flattened_masks_stack.shape}')
print(f'shape flatened stack: {flattened_masks_stack.max()}')

shape flatened stack: (25, 2400, 2400)
shape flatened stack: 3.0


In [26]:
image_stack=image_stack[:5]
flattened_masks_stack=flattened_masks_stack[:5]

In [30]:
import numpy as np

def split_images_to_quarters(image_stack):
    """
    Splits each image in the input stack into four equal-sized quarters.
    
    Args:
        image_stack (numpy.ndarray): A 3D numpy array representing a stack of images.
                                   The shape should be (num_images, height, width).
    
    Returns:
        numpy.ndarray: A 3D numpy array with the images split into quarters.
                      The new shape will be (4 * num_images, height // 2, width // 2).
    """
    # Get the shape of the input image stack
    num_images, height, width = image_stack.shape
    
    # Split each image into four equal-sized quarters
    quarters = np.empty((4 * num_images, height // 2, width // 2), dtype=image_stack.dtype)
    
    for i in range(num_images):
        # Split the current image into four quarters
        img = image_stack[i]
        q1 = img[:height//2, :width//2]
        q2 = img[:height//2, width//2:]
        q3 = img[height//2:, :width//2]
        q4 = img[height//2:, width//2:]
        
        # Store the quarters in the output array
        quarters[4*i] = q1
        quarters[4*i+1] = q2
        quarters[4*i+2] = q3
        quarters[4*i+3] = q4
    
    return quarters


In [37]:
image_stack = split_images_to_quarters(image_stack)
flattened_masks_stack = split_images_to_quarters(flattened_masks_stack)

flattened_masks_stack=flattened_masks_stack[:5]
image_stack=image_stack[:5]


In [38]:
from sklearn.model_selection import train_test_split


random_state = np.random.randint(1, 100)
train_size = np.random.uniform(0.6, 0.8)

images_train, images_test, labels_train, labels_test = train_test_split(
    image_stack, flattened_masks_stack, test_size=1 - train_size, random_state=random_state
)

print("Image stack shape:")
print(f"Training set: {images_train.shape}")
print(f"Test set: {images_test.shape}")

print("\nMask stack shape:")
print(f"Training set: {labels_train.shape}")
print(f"Test set: {labels_test.shape}")

Image stack shape:
Training set: (3, 300, 300)
Test set: (2, 300, 300)

Mask stack shape:
Training set: (3, 300, 300)
Test set: (2, 300, 300)


In [41]:
import atomai as aoi


model = aoi.models.Segmentor(nb_classes=3)  # uses UNet by default


model.fit(images_train, labels_train, images_test, labels_test, batch_size=1, print_loss=1,# training data (numpy arrays)
          training_cycles=30, compute_accuracy=True, swa=True # training parameters
    ) 

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 75 but got size 74 for tensor number 1 in the list.

In [ ]:
t.save_model("my_model")
